# Refactoring exec_pool as a core engine
We are refactoring execution of a contract pool by building a new `async exec_pool` function.

`async exec_pool`:
1. processes sets of contracts to run specific algos 
2. with controlled concurrency 
3. with an option to produce df outputs
   - which provides the capability to checkpoint to a pickle file...
   - ... thereby `re-start` from near a point of failure


In [ ]:
MARKET = 'NSE'

In [ ]:
import sys
import pathlib
import pandas as pd
import yaml
import asyncio

from ib_insync import IB, util, Option, MarketOrder
from typing import Callable, Coroutine

In [ ]:
# Specific to Jupyter. Will be ignored in IDE / command-lines
import IPython as ipy
if ipy.get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
    import nest_asyncio
    nest_asyncio.apply()
    util.startLoop()
    pd.options.display.max_columns = None

In [ ]:
# Get capability to import programs from `asyncib` folder
cwd = pathlib.Path.cwd() # working directory from where python was initiated
FSPATH = cwd.joinpath('data') # path to store data files
LOGPATH = FSPATH # path to store log files

IBPATH = cwd.parent.parent.joinpath('asyncib') # where ib programs are stored

# append IBPATH to import programs.
if str(IBPATH) not in sys.path:  # Convert it to string!
    sys.path.append(str(IBPATH)) 

In [ ]:
# local imports
from base import chains, unds, qualify, prices, margins

In [ ]:
# Get the yaml config for HOST, PORT, CID
with open(IBPATH.joinpath('var.yml')) as fi:
    data = yaml.safe_load(fi)
    
HOST = data["COMMON"]["HOST"]
PORT = data[MARKET.upper()]["PORT"]
CID = data["COMMON"]["CID"]

# Set log file
util.logToFile(FSPATH.joinpath('./engine.log'), level=30)

# The ``async exec_pool`` algo
### with concurrency control and post-processing checkpoints

In [ ]:
async def executeAsync(ib: IB(),
                       algo: Callable[..., Coroutine],  # coro name
                       cts: list, # list of contracts
                       post_process: Callable[[set, pathlib.Path, str], pd.DataFrame]=None, # If checkpoint is needed
                       FSPATH: pathlib.Path=None, # Necessary for post_process
                       CONCURRENT: int=40, # adjust to prevent overflows
                       TIMEOUT: None=None, # if None, no progress messages shown
                       OP_FILENAME: str='', # output file name
                       **kwargs, # keyword inputs for algo
                       ):
    
    tasks = set()
    results = set()
    remaining = tuple(cts)
    
    # Determine unique names for tasks
    try:
        remaining[0].symbol
        
    except AttributeError: # It is a (contract, order) tuple for margin algo!
        ct_name="c[0].symbol+c[0].lastTradeDateOrContractMonth[-4:]+c[0].right+str(c[0].strike)+'..'"
        
    else: # for all algos, except margin algo
        ct_name="c.symbol+c.lastTradeDateOrContractMonth[-4:]+c.right+str(c.strike)+'..'"

    # Get the results
    while len(remaining):
    
        # Tasks limited by concurrency
        if len(remaining) <= CONCURRENT:
            tasks.update(asyncio.create_task(algo(ib, c, **kwargs), name=eval(ct_name)) for c in remaining)
        else:
            tasks.update(asyncio.create_task(algo(ib, c, **kwargs), name=eval(ct_name)) for c in list(remaining)[:CONCURRENT])

        # Execute tasks
        while len(tasks):

            done, tasks = await asyncio.wait(tasks,
                                            timeout=TIMEOUT,
                                            return_when=asyncio.ALL_COMPLETED)

            # Remove dones from remaining
            done_names = [d.get_name() for d in done]
            remaining = [c for c in remaining if eval(ct_name) not in done_names]
            
            # Update results and checkpoint
            results.update(done)
            
            # Checkpoint the results
            if post_process:
                output = post_process(results, FSPATH, OP_FILENAME)
            else:
                output = results
            
            if TIMEOUT:
                print(f'\nCompleted {done_names[:2]} {len(results)} out of {len(cts)} .. remaining {[eval(ct_name) for c in remaining][:2]}')
    
    return output


def save_df(results: set, FSPATH: pathlib.Path, file_name: str='') -> pd.DataFrame():

    if results:
        df = pd.concat([r.result() for r in results if r], ignore_index=True)
        if file_name:
            df.to_pickle(FSPATH.joinpath(file_name))
    else:
        df = pd.DataFrame([]) # results are not yet ready!
    return df

## Testing `async exec_pool` algo

In [ ]:
# Get symlots
IBDATAPATH = IBPATH.joinpath('data', MARKET.lower())
df_symlots = pd.read_pickle(IBDATAPATH.joinpath('df_symlots.pkl'))

und_cts = df_symlots.contract

### Uncomment for !!! DATA LIMITING underlying contracts
# und_cts = df_symlots.contract[:50].to_list()

### Get underlyings

### Make the chains

### Qualify ready-made options

### Prepare and qualify fresh set of options (MEGA)
* Run this code if the ENTIRE set of options available in the market for ALL options

### Get the price of qualified options

### Prepare cos for margins from qualified options

In [ ]:
df_symlots = pd.read_pickle(IBDATAPATH.joinpath('df_symlots.pkl'))
df_raw_opts = pd.read_pickle(IBDATAPATH.joinpath('df_qopts.pkl'))

if MARKET == 'NSE':
    df_raw_opts['expiryM'] = df_raw_opts.expiry.apply(
        lambda d: d[:4] + '-' + d[4:6])
    cols1 = ['symbol', 'expiryM']
    df_raw_opts = df_raw_opts.set_index(cols1).join(
        df_symlots[cols1 + ['lot']].set_index(cols1)).reset_index()
    df_raw_opts = df_raw_opts.drop('expiryM', 1)
else:
    df_raw_opts['lot'] = 100

# ... build cos (contract, orders)
opts = df_raw_opts.contract.to_list()
orders = [MarketOrder('SELL', lot / lot) if MARKET.upper() ==
          'SNP' else MarketOrder('SELL', lot) for lot in df_raw_opts.lot]
cos = [(c, o) for c, o in zip(opts, orders)]

In [ ]:
# cos = cos[:500] # !!! DATA LIMITER

### Get option margins

In [ ]:
%%time
with IB().connect(HOST, PORT, CID) as ib:
    df_margins = ib.run(executeAsync(ib=ib, algo=margins, cts=cos, 
                                  CONCURRENT=200, TIMEOUT=5.0,
                                  post_process=save_df, FSPATH=FSPATH, OP_FILENAME='df_margins.pkl',))

### Get option prices

In [ ]:
%%time
with IB().connect(HOST, PORT, CID) as ib:
    df_price = ib.run(executeAsync(ib=ib, algo=prices, cts=opts, 
                                  CONCURRENT=200, TIMEOUT=10.0,
                                  post_process=save_df, FSPATH=FSPATH, OP_FILENAME='df_optprices.pkl',))

In [ ]:
df = df_price[~df_price.price.isnull()]

In [ ]:
df1 = df.set_index('conId').join(df_margins[['conId', 'margin', 'lot', 'comm']].set_index('conId')).reset_index()

In [ ]:
df1[~df1.iv.isnull()]